<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

In [2]:
!pip install langdetect

In [7]:
#!pip3 install convokit
#!python3 -m spacy download en

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Preprocessing

In [4]:
import numpy as np
import json
from anytree import Node, RenderTree

In [5]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [6]:
def extract_files_list(file_list_path):
    files_name = []
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                files_name.append(file_path)
    return files_name

In [7]:
data_files_name = extract_files_list(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")

In [8]:
def make_submission_file_path(file_name):
    return f"{base_path}submissions_{file_name}.json"

In [9]:
def make_comment_file_path(file_name):
    return f"{base_path}comments_{file_name}.json"

In [10]:
subreddits = set()

In [11]:
def extract_submissions_text_with_filter(data_files_name):
    submissions_dic = {}
    over_18_count = 0
    for file_name in data_files_name:
        file_path = make_submission_file_path(file_name)
        with open(file_path, 'r') as file:
            lines_sub = file.readlines()
        for l in lines_sub:
            obj = json.loads(l)
            if obj['over_18'] == False:
                submissions_dic[obj['id']]  = obj['title'] + obj['selftext']
                subreddits.add(obj['subreddit'])
            else:
                over_18_count += 1
    print("over 18 count ", over_18_count)
    return submissions_dic

In [12]:
subsmissions = extract_submissions_text_with_filter(data_files_name)

over 18 count  66876


In [13]:
len(subsmissions)

96333

In [14]:
import json

def count_comments_in_file(file_path):
    count = 0
    with open(file_path, 'r') as json_file:
        lines = json_file.readlines()
        for line in lines:
            data = json.loads(line)
            count += len(data.get('comments', []))
    return count

def total_comments_in_files(file_list_path):
    total_comments = 0
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            print(line)
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                total_comments += count_comments_in_file(f"{base_path}comments_{file_path}.json")
    return total_comments

# Replace 'file_list.txt' with the path to your text file containing the list of file names
#total_comments = total_comments_in_files(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")
#print(f'Total number of comments in the files: {total_comments}')

In [15]:
def load_comments_with_filter(files_name, submmissions):
    # Create a dictionary to store the comment nodes by their IDs
    conversations_dic = {}
    for file_name in data_files_name:
        comment_file = make_comment_file_path(file_name)
        with open(comment_file, 'r') as file:
            lines = file.readlines()

        for line in lines:
            comment_nodes = {}
            json_obj = json.loads(line)
            submission_id = json_obj['submission_id']
            comments = json_obj['comments']
            if len(comments) == 0:
              print("empty")
              continue
            if submission_id not in subsmissions:
                #submission has removed in preprocessing
                continue
            flat_comments_list = []
            parent_child_pairs = []
            # Create root node for the submission
            submission_text = subsmissions[submission_id]
            root = Node(submission_id, body = submission_text)
            comment_nodes[submission_id] = root
            conversation_text = submission_text
            flat_comments_list.append({'id': submission_id, 'body': submission_text, 'author':None, 'reply_to':None, 'conversation_id':submission_id})
            # Create child nodes for the comments
            for comment in comments:
                # Check if comment is from a bot and ignore it if it is
                if "I am a bot, and this action was performed automatically" in comment['body']:
                    continue

                comment_id = comment['id']
                author = comment.get('author_fullname', 'unknown')
                parent_id = comment['parent_id'].split('_', 1)[-1]
                body = comment['body']
                comment_node = Node(comment_id, body=body)

                if parent_id in comment_nodes:
                    flat_comments_list.append({'id': comment_id, 'body': body, 'author':author, 'reply_to':parent_id, 'conversation_id':submission_id})
                    parent_node = comment_nodes[parent_id]
                    parent_child_pairs.append({
                        'comment': parent_node.body,
                        'reply': body,
                        'comment_id': parent_node.name,
                        'reply_id': comment_id,
                        'global_id': parent_node.name + "_" + comment_id
                    })
                    comment_node.parent = parent_node
                    conversation_text += "/n"+ body
                    comment_nodes[comment_id] = comment_node
                else:
                    print("reply to bot")
            # Print the tree structure
            # print(RenderTree(root))
            conversations_dic[submission_id] = {
                'tree': root,
                'full_text': conversation_text,
                'pairs': parent_child_pairs,
                'comments': flat_comments_list
            }
            #for parent in root.children:
            #  for child in parent.children:
            #       parent_child_pairs.append((parent.body, child.body))

            # Print the parent-child pairs
            #for pair in parent_child_pairs:
            #   print(f"Parent: {pair[0]}, Child: {pair[1]}")

            # Print the tree structure
            #for pre, fill, node in RenderTree(root):
            #   print(f"{pre}{node.name}")

        #print(RenderTree(conversations_tree[2555]))
    return conversations_dic

In [16]:
conversations_dic = load_comments_with_filter(data_files_name, subsmissions)

empty
empty
empty
empty
empty
empty
reply to bot
empty
empty
empty
empty
empty
reply to bot
reply to bot
empty
empty
reply to bot
empty
empty
empty
empty
reply to bot
reply to bot
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
reply to bot
empty
empty
empty
empty
reply to bot
empty
empty
empty
empty
empty
empty
reply to bot
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
reply to bot
empty
empty
empty
reply to bot
empty
empty
empty
empty
empty
reply to bot
empty
empty
empty
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
reply to bot
empty
empty
empty
empty
empty
empty
empty
empty
reply to bot
empty
empty
empty
empty
empty
empty
empty
reply to bot
reply to bot
reply to bot
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
reply to bot


In [17]:
len(conversations_dic)

39784

In [28]:
def filter_empty_conversations(conversations_dic):
  non_empty_conversations = {}
  for id in conversations_dic:
    new_pairs = []
    pairs = conversations_dic[id]['pairs']
    for pair in pairs:
      if not(pair['comment'] == '[deleted]' or pair['reply'] == '[deleted]' or pair['comment'].endswith('[removed]')):
        new_pairs.append(pair)
    if len(new_pairs) > 0:
      conversations_dic[id]['pairs'] = new_pairs
      non_empty_conversations[id] = conversations_dic[id]
  return non_empty_conversations


In [29]:
non_empty_conversations = filter_empty_conversations(conversations_dic)

In [30]:
len(conversations_dic)

39784

In [31]:
len(non_empty_conversations)

35393

In [35]:
from langdetect import detect, LangDetectException

def filter_non_english(conversations_dic):
    english_conversations = {}
    for id in conversations_dic:
        try:
            language = detect(conversations_dic[id]['full_text'])
            if language == 'en':
                english_conversations[id] = conversations_dic[id]
        except LangDetectException as e:
            # Handle the exception and print the problematic input
            # print(f"Language detection error for comment: {conversations_dic[id]['full_text']}")
            continue
    return english_conversations

In [36]:
english_conversations = filter_non_english(non_empty_conversations)

In [37]:
len(english_conversations)

33574

In [ ]:
import random


# Define the sample size (e.g., 3 for a sample of 3 elements)
sample_size = 20

# Use random.sample to get a random sample of the specified size
random_sample = random.sample(submissions_dic, sample_size)

random_sample

TypeError: ignored

In [ ]:
parent_child_pairs[:10]

[{'comment': "Curse you Hurrican IanTargets and Walmarts near me didn't get any ghouls in due to the hurricane. Hunting today was extremely disheartening, I guess I'll just have to keep checking.",
  'reply': "I've been out looking for Ghoulia today, good to know why I couldn't find her.",
  'comment_id': 'xsyg0d',
  'reply_id': 'iqmwd8f',
  'global_id': 'xsyg0d_iqmwd8f'},
 {'comment': "Curse you Hurrican IanTargets and Walmarts near me didn't get any ghouls in due to the hurricane. Hunting today was extremely disheartening, I guess I'll just have to keep checking.",
  'reply': 'yeah, fl really got screwed over. was hoping to find my ghouls today but not even a peep, and the workers didn’t even know when they’d be getting stock. i just hope they go online soon',
  'comment_id': 'xsyg0d',
  'reply_id': 'iqn4okm',
  'global_id': 'xsyg0d_iqn4okm'},
 {'comment': "Curse you Hurrican IanTargets and Walmarts near me didn't get any ghouls in due to the hurricane. Hunting today was extremely di

In [ ]:
removed_comments = []
count = 0
for comment in flat_comments_list:
  if comment['body'] == '[deleted]' or comment['body'] == ['removed']:
    count += 1
count

418

In [ ]:
from langdetect import detect, LangDetectException

non_english = []
for conversation_text in conversation_full_text:
    try:
        language = detect(conversation_text['text'])
        if language != 'en':
            non_english.append(conversation_text)
    except LangDetectException as e:
        # Handle the exception and print the problematic input
        # print(f"Language detection error for comment: {comment['body']}")
        continue

In [ ]:
len(conversation_full_text)

2885

In [ ]:
len(non_english)

226

In [ ]:
non_english[0:100]

[{'text': '26 m usa. dm for sc. kinky++ freaks+++ longterm++', 'id': 'xsyfyw'},
 {'text': 'Gente de Reddit que vive/ha vivido en España, Cuales fueron vuestras expectativas y la realidad cuando vinieron a vivir en España? La realidad superó vuestras expectativas? O la realidad fue "peor" a vuestras expectativas?/nEso dependerá mucho de las expectativas, tienes un ejemplo o algún contexto?',
  'id': 'xsyfyg'},
 {'text': 'BRUNETTE STRIPPING', 'id': 'xsyfwz'},
 {'text': 'Slide in', 'id': 'xsyfu5'},
 {'text': 'Deeper', 'id': 'xsyftp'},
 {'text': 'Boop tha kitty./nVulkan Boops!  \n\n*stomp stomp*', 'id': 'xsyfre'},
 {'text': 'Dua Lipa/nStunning 🔥', 'id': 'xsyfqm'},
 {'text': 'Yveltal 6870 8865 8721 / 1456 3069 6377 adding 10/ngucci1goat/nHamster618',
  'id': 'xsyfq7'},
 {'text': "Comment s’appelle les ennemis qui donnent beaucoup d’expériences dans votre licence préférée ? Dans dragon quest ce sont les monstres de métal/nClassique leuphorie/nSalut, c’est quel dragon quest celui ci?/nEn l'oc

In [ ]:
len(flat_comments_list)

36685

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=1474eecd4c375d71e655204e051baeeca62a85f1da83b2b1a988788713fcb941
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect

text = "This is a sample English text."
language = detect(text)
print(language)  # Output: 'en' for English

en


In [ ]:
import random


# Define the sample size (e.g., 3 for a sample of 3 elements)
sample_size = 1

# Use random.sample to get a random sample of the specified size
random_sample = random.sample(conversations_tree, sample_size)
from anytree import PreOrderIter

# Assuming 'root' is the root node of the conversation tree
for node in PreOrderIter(random_sample[0]):
    if node.parent is not None:
        print(f"{node.name}: {node.body}")

random_sample

iqmyeuo: Hey, we launched a new music-scanning feature at Soundslice just a few days ago ([see here](https://www.soundslice.com/blog/226/pdf-and-photo-scanning-beta/)). It uses artificial intelligence to convert PDFs/images into editable sheet music.

I'm happy to run a PDF through it for you if you'd like — just send me a DM!


[Node('/xsyewi', body="Has Anyone Tried Scanscore ?I've been trying to find a pdf scanner app or program that can scan sheet music and turn it into a **musicxml** format. I came across Scanscore, but I don't if it's any good. anyone in this sub try it? would you recommend it?")]

In [ ]:
flat_comments_list[:10]

[{'id': 'xsyg0d',
  'body': "Curse you Hurrican IanTargets and Walmarts near me didn't get any ghouls in due to the hurricane. Hunting today was extremely disheartening, I guess I'll just have to keep checking.",
  'author': None,
  'reply_to': None,
  'conversation_id': 'xsyg0d'},
 {'id': 'iqmwd8f',
  'body': "I've been out looking for Ghoulia today, good to know why I couldn't find her.",
  'author': 't2_57iup',
  'reply_to': 'xsyg0d',
  'conversation_id': 'xsyg0d'},
 {'id': 'iqn4okm',
  'body': 'yeah, fl really got screwed over. was hoping to find my ghouls today but not even a peep, and the workers didn’t even know when they’d be getting stock. i just hope they go online soon',
  'author': 't2_mwcj2qdc',
  'reply_to': 'xsyg0d',
  'conversation_id': 'xsyg0d'},
 {'id': 'iqpb78k',
  'body': 'Ian is such a shit name for a hurricane too, like if you’re going to ruin our doll hunting experience at least have a cool name',
  'author': 't2_2ey2bons',
  'reply_to': 'xsyg0d',
  'conversation

In [ ]:
!pip install  kili

In [ ]:
import os

from kili.client import Kili
kili = Kili(api_key='6acd489f-9746-4ac4-9650-54e2d9c9faba')
interface = {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"Related": {"name": "is related"}, "UnRelated": {"name": "is not related"}},
                "input": "radio",
            },
        }
    }
}

project = kili.create_project(
    title="Testing ...",
    description="Project Description",
    input_type="TEXT",
    json_interface=interface,
)
project_id = project['id']
print("Project ID: ", project_id)
assets = kili.append_many_to_dataset(
    project_id=project_id,
    content_array = [(d['comment'] + d['reply']) for d in parent_child_pairs[:5]],
    external_id_array = [d['global_id'] for d in parent_child_pairs[:5]]
)

Project ID:  clo71jb7a1nxb079tdn1zh7hn


In [ ]:
from anytree import Node, RenderTree

# Create a list to store parent-child pairs
parent_child_pairs = []

# Access child nodes from the parent node and add the pairs to the list
for parent in root.children:
    for child in parent.children:
        parent_child_pairs.append((parent.name, child.name))

# Print the parent-child pairs
for pair in parent_child_pairs:
    print(f"Parent: {pair[0]}, Child: {pair[1]}")

# Print the tree structure
for pre, fill, node in RenderTree(root):
    print(f"{pre}{node.name}")


Parent: iqmum2s, Child: iqmunnz
Parent: iqmum2s, Child: iqmuns8
Parent: iqon7tc, Child: iqrmqmy
xsy76f
├── iqmum2s
│   ├── iqmunnz
│   └── iqmuns8
├── iqob6q0
└── iqon7tc
    └── iqrmqmy
        └── iqsvfoq


In [ ]:
import json

for comment in flat_comments_list:


Data has been prepared for Kili labeling.


In [ ]:
formatted_data = []
for comment in flat_comments_list:
    formatted_comment = {
        "row_data": comment['body'],
        "global_key": comment['id'],
        "media_type": "TEXT",
        "metadata_fields": [],
        "attachments": []
    }
    formatted_data.append(formatted_comment)

In [ ]:
with open(base_path +"flat_comments_list.json", "w") as json_file:
    json.dump(flat_comments_list, json_file, indent=2)

print("Data has been written to formatted_comments.json")


Data has been written to formatted_comments.json


In [ ]:
conversations_tree.index(0)

ValueError: ignored

In [ ]:
def flatten_tree(node, flattened_tree):
    flattened_tree.append(node)
    for child in node.children:
        flatten_tree(child, flattened_tree)

In [ ]:
flattened_tree = []
flatten_tree(root, flattened_tree)

In [ ]:
flatten_tree

<function __main__.flatten_tree(node, flattened_tree)>

In [ ]:
# there is node without body!

In [ ]:
from convokit import Corpus, Utterance, Speaker
# Create a Corpus object from the flattened tree.
corpus = Corpus(utterances=[
        Utterance(id=node['id'], text=node['body'], speaker=Speaker(id = node['author']), reply_to=node['reply_to'], conversation_id=node['conversation_id'])
        for node in flat_comments_list
])

In [ ]:
corpus = corpus.fit(corpus)

AttributeError: ignored

In [ ]:
counter = 0
for convo in corpus.iter_conversations():
    counter+=1
print(counter)

2940


In [ ]:
corpus.print_summary_stats()

Number of Speakers: 20381
Number of Utterances: 37647
Number of Conversations: 2940


In [ ]:
from convokit.text_processing import TextParser
parser = TextParser(verbosity=100)
corpus = parser.transform(corpus)

100/37647 utterances processed
200/37647 utterances processed
300/37647 utterances processed
400/37647 utterances processed
500/37647 utterances processed
600/37647 utterances processed
700/37647 utterances processed
800/37647 utterances processed
900/37647 utterances processed
1000/37647 utterances processed
1100/37647 utterances processed
1200/37647 utterances processed
1300/37647 utterances processed
1400/37647 utterances processed
1500/37647 utterances processed
1600/37647 utterances processed
1700/37647 utterances processed
1800/37647 utterances processed
1900/37647 utterances processed
2000/37647 utterances processed
2100/37647 utterances processed
2200/37647 utterances processed
2300/37647 utterances processed
2400/37647 utterances processed
2500/37647 utterances processed
2600/37647 utterances processed
2700/37647 utterances processed
2800/37647 utterances processed
2900/37647 utterances processed
3000/37647 utterances processed
3100/37647 utterances processed
3200/37647 uttera

In [ ]:
from convokit import politenessStrategies
ps = politenessStrategies.PolitenessStrategies(verbose = 1000)
corpus = ps.fit_transform(corpus=corpus)

1000/37647 utterances processed
2000/37647 utterances processed
3000/37647 utterances processed
4000/37647 utterances processed
5000/37647 utterances processed
6000/37647 utterances processed
7000/37647 utterances processed
8000/37647 utterances processed
9000/37647 utterances processed
10000/37647 utterances processed
11000/37647 utterances processed
12000/37647 utterances processed
13000/37647 utterances processed
14000/37647 utterances processed
15000/37647 utterances processed
16000/37647 utterances processed
17000/37647 utterances processed
18000/37647 utterances processed
19000/37647 utterances processed
20000/37647 utterances processed
21000/37647 utterances processed
22000/37647 utterances processed
23000/37647 utterances processed
24000/37647 utterances processed
25000/37647 utterances processed
26000/37647 utterances processed
27000/37647 utterances processed
28000/37647 utterances processed
29000/37647 utterances processed
30000/37647 utterances processed
31000/37647 utteran

In [ ]:
ps.summarize(corpus=corpus)

feature_politeness_==Please==                 0.015220
feature_politeness_==Please_start==           0.028183
feature_politeness_==HASHEDGE==               0.198528
feature_politeness_==Indirect_(btw)==         0.000186
feature_politeness_==Hedges==                 0.084841
feature_politeness_==Factuality==             0.058889
feature_politeness_==Deference==              0.014822
feature_politeness_==Gratitude==              0.041783
feature_politeness_==Apologizing==            0.008739
feature_politeness_==1st_person_pl.==         0.068956
feature_politeness_==1st_person==             0.251999
feature_politeness_==1st_person_start==       0.201291
feature_politeness_==2nd_person==             0.233724
feature_politeness_==2nd_person_start==       0.053789
feature_politeness_==Indirect_(greeting)==    0.009669
feature_politeness_==Direct_question==        0.050416
feature_politeness_==Direct_start==           0.088188
feature_politeness_==HASPOSITIVE==            0.413393
feature_po

In [ ]:
from anytree import Node, find
corpus.get_utterance("iqp0hc8")
target_node = find(conversations_tree, lambda node: node.name == "iqp0hc8")
print(target_node.body)

AttributeError: ignored

In [ ]:
corpus.get_conversation("xsyg0d").meta

ConvoKitMeta({})

In [ ]:
def calculate_conversation_score(dataset_with_metrics_score, metrics_weights):
  i = 2

In [ ]:
file_path = base_path + 'train.tsv.tar.gz'

In [ ]:
import tarfile
import gzip
import pandas as pd

# Specify the path to the .tsv.tar.gz file

# Open the .tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract the .tsv.gz file from the .tar.gz archive
    tsv_file = tar.getmembers()[0]
    file_extension = tsv_file.name.split('.')[-1]

    # Check if the file is compressed
    if file_extension == 'gz':
        # Open the .tsv.gz file
        with gzip.open(tar.extractfile(tsv_file), 'rt') as tsv:
            # Read the lines and store them in a list
            lines = [line.strip() for line in tsv]

    else:
        # Open the uncompressed .tsv file
        with tar.extractfile(tsv_file) as tsv:
            # Read the lines and store them in a list
            lines = [line.decode().strip() for line in tsv]

# Create a DataFrame from the lines
df = pd.DataFrame([line.split('\t') for line in lines])

# Optionally, you can set the column names based on the first row
df.columns = df.iloc[0]
df = df[1:]  # Remove the first row (column names)

# Print the DataFrame
print(df)


KeyboardInterrupt: ignored

In [ ]:
import tarfile
import gzip
import pandas as pd

# Specify the path to the .tsv.tar.gz file
#file_path = 'path/to/your/file.tsv.tar.gz'

# Specify the number of lines to read at a time
batch_size = 1000

# Open the .tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract the .tsv.gz file from the .tar.gz archive
    tsv_file = tar.getmembers()[0]
    file_extension = tsv_file.name.split('.')[-1]

    # Check if the file is compressed
    if file_extension == 'gz':
        # Open the .tsv.gz file
        with gzip.open(tar.extractfile(tsv_file), 'rt') as tsv:
            # Read the lines in batches
            lines = []
            for line in tsv:
                lines.append(line.strip())
                if len(lines) == batch_size:
                    # Process the lines and create a DataFrame
                    df = pd.DataFrame([line.split('\t') for line in lines])

                    # Optionally, set the column names based on the first row
                    df.columns = df.iloc[0]
                    df = df[1:]  # Remove the first row (column names)

                    # Do further processing with the DataFrame
                    # ...

                    # Clear the lines list to free memory
                    lines = []

            # Process the remaining lines
            if lines:
                df = pd.DataFrame([line.split('\t') for line in lines])
                df.columns = df.iloc[0]
                df = df[1:]

    else:
        # Open the uncompressed .tsv file
        with tar.extractfile(tsv_file) as tsv:
            # Read the lines in batches
            lines = []
            for line in tsv:
                lines.append(line.decode().strip())
                if len(lines) == batch_size:
                    # Process the lines and create a DataFrame
                    df = pd.DataFrame([line.split('\t') for line in lines])

                    # Optionally, set the column names based on the first row
                    df.columns = df.iloc[0]
                    df = df[1:]  # Remove the first row (column names)

                    # Do further processing with the DataFrame
                    # ...
                    print(df.columns)
                    # Clear the lines list to free memory
                    lines = []

            # Process the remaining lines
            if lines:
                df = pd.DataFrame([line.split('\t') for line in lines])
                df.columns = df.iloc[0]
                df = df[1:]

# Print the DataFrame
#print(df)
